# Decision Tree Classifier (Explanation Baseline)

We consider the DecisionTree Visualization the baseline of our explanation task. Our goal is to be more clear in our decision with respect to DecisionTree.

We use the DecisionTreeClassifier of sklearn library, you can find more info at https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

In [49]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, recall_score
from sklearn import tree
import matplotlib.pyplot as plt

In [27]:
clf = DecisionTreeClassifier(random_state=23)

In [28]:
# data paths
DATASET_PATH = "../data/migraine_experiments.xls"

In [29]:
data = pd.read_excel(DATASET_PATH, skiprows=1)

# columns name cleaner 
d_columns_temp = data.columns
d_columns = []

for c in d_columns_temp:
    c_cleaned = c.split("\n")[0].lower()
    d_columns.append(c_cleaned)

data.columns = d_columns

# features groups
groups_temp = list(pd.read_excel(DATASET_PATH).columns.values)
groups = []

for index in range(len(groups_temp)):
    if "NA:" not in groups_temp[index] and "Unnamed:" not in groups_temp[index]:
        groups.append(groups_temp[index])
        
data.head()

,cod.,età (all'arruolamento),genere,menopausa,menarca (età),emicrania,diagnosi,lato,onset (età),familiarità,...,bmi,fumo si=1/no=0,alcool si=1/no=0,caffè si=1/no=0,oc/hrt si=1/no=0,familiarità per cardiopatia,ipertensione arteriosa,ima si=1/no=0,dislipidemia si=1/no=0,diabete si=1/no=0
0,BB12592,42.0,1,0.0,NaN,2,ESA,1.0,20.0,1.0,...,31.2,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,SOD001,42.0,1,0.0,15.5,3,EC,2.0,30.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SOD002,38.0,1,0.0,12.5,3,EC+MOH,1.0,15.0,1.0,...,23.7,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,SOD003,44.0,0,NaN,NaN,3,EC+MOH,1.0,16.0,1.0,...,NaN,0.0,0.0,1.0,NaN,1.0,0.0,0.0,0.0,0.0
4,SOD004,72.0,0,NaN,NaN,3,EC,1.0,41.0,1.0,...,22.8,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0


In [30]:
CLASSES = data["emicrania"].unique()
CLASSES

array([2, 3, 1, 4, 5], dtype=int64)

In [31]:
data.head()

,cod.,età (all'arruolamento),genere,menopausa,menarca (età),emicrania,diagnosi,lato,onset (età),familiarità,...,bmi,fumo si=1/no=0,alcool si=1/no=0,caffè si=1/no=0,oc/hrt si=1/no=0,familiarità per cardiopatia,ipertensione arteriosa,ima si=1/no=0,dislipidemia si=1/no=0,diabete si=1/no=0
0,BB12592,42.0,1,0.0,NaN,2,ESA,1.0,20.0,1.0,...,31.2,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,SOD001,42.0,1,0.0,15.5,3,EC,2.0,30.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SOD002,38.0,1,0.0,12.5,3,EC+MOH,1.0,15.0,1.0,...,23.7,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,SOD003,44.0,0,NaN,NaN,3,EC+MOH,1.0,16.0,1.0,...,NaN,0.0,0.0,1.0,NaN,1.0,0.0,0.0,0.0,0.0
4,SOD004,72.0,0,NaN,NaN,3,EC,1.0,41.0,1.0,...,22.8,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0


In [32]:
data.describe()

,età (all'arruolamento),genere,menopausa,menarca (età),emicrania,lato,onset (età),familiarità,uas,emicrania mestruale,...,bmi,fumo si=1/no=0,alcool si=1/no=0,caffè si=1/no=0,oc/hrt si=1/no=0,familiarità per cardiopatia,ipertensione arteriosa,ima si=1/no=0,dislipidemia si=1/no=0,diabete si=1/no=0
count,1038.000000,1039.000000,850.000000,266.000000,1039.000000,1037.000000,1037.000000,774.000000,1037.000000,464.000000,...,722.000000,495.000000,495.000000,494.000000,394.000000,491.000000,495.000000,495.00000,495.000000,493.000000
mean,42.315029,0.820019,0.243529,12.204887,1.607315,1.533462,19.481196,0.713178,0.509161,0.400862,...,23.339515,0.272727,0.155556,0.831984,0.164975,0.260692,0.103030,0.00404,0.125253,0.018256
std,12.814378,0.384356,0.429465,1.225644,0.896248,0.717358,10.407938,0.452570,0.500157,0.490602,...,3.572552,0.454803,0.362800,0.374260,0.371630,0.439460,0.304306,0.06350,0.331340,0.134010
min,13.000000,0.000000,0.000000,8.500000,1.000000,1.000000,-1.000000,0.000000,0.000000,0.000000,...,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,33.000000,1.000000,0.000000,11.000000,1.000000,1.000000,12.000000,0.000000,0.000000,0.000000,...,21.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
50%,42.000000,1.000000,0.000000,12.000000,1.000000,1.000000,17.000000,1.000000,1.000000,0.000000,...,23.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
75%,51.000000,1.000000,0.000000,13.000000,2.000000,2.000000,25.000000,1.000000,1.000000,1.000000,...,25.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.00000,0.000000,0.000000
max,78.000000,1.000000,1.000000,16.000000,5.000000,4.000000,73.000000,1.000000,1.000000,1.000000,...,46.800000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000


In [33]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1039 entries, 0 to 1038
Data columns (total 40 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   cod.                                   1039 non-null   object 
 1   età (all'arruolamento)                 1038 non-null   float64
 2   genere                                 1039 non-null   int64  
 3   menopausa                              850 non-null    float64
 4   menarca (età)                          266 non-null    float64
 5   emicrania                              1039 non-null   int64  
 6   diagnosi                               1039 non-null   object 
 7   lato                                   1037 non-null   float64
 8   onset (età)                            1037 non-null   float64
 9   familiarità                            774 non-null    float64
 10  frequenza attacchi al mese             1031 non-null   object 
 11  uas 

## Fill NaN data

In [34]:
import random

def compute_mode(a):
    elements = list(a.fillna(-1).unique())
    arr_count = np.zeros(len(elements))
    for el in list(elements):
        if type(el) != str and el != np.nan:
            arr_count[elements.index(el)] += 1
    return elements[np.argmax(arr_count)]
    

def fill_binary_columns(list_of_indexes):
    for index in list_of_indexes:
        mode = compute_mode(data[data.columns[index]])
        data[data.columns[index]] = data[data.columns[index]].replace("/",mode)
        data[data.columns[index]] = data[data.columns[index]].replace("O",mode)
        data[data.columns[index]] = data[data.columns[index]].fillna(mode)
  
        
def fill_categorical_columns(list_of_indexes):
    for index in list_of_indexes:
        mode = compute_mode( data[data.columns[index]])
        data[data.columns[index]] = data[data.columns[index]].replace("/",mode)
        data[data.columns[index]] = data[data.columns[index]].replace("O",mode)
        data[data.columns[index]] = data[data.columns[index]].fillna(mode)


        
def fill_range_columns(list_of_indexes):
    for index in list_of_indexes:
        mode = compute_mode( data[data.columns[index]])        
        data[data.columns[index]] = data[data.columns[index]].replace("/",mode)
        data[data.columns[index]] = data[data.columns[index]].replace("O",mode)
        data[data.columns[index]] = data[data.columns[index]].fillna(mode)
        data[data.columns[index]] = data[data.columns[index]].apply(categorical_to_integer, args=(index,))



def get_unique_range_value(index):
    return list(data[data.columns[index]].unique())
    
def categorical_to_integer(row, c):
    c = get_unique_range_value(c)
    return  c.index(row)

def build_target_column(row):
    return list(CLASSES).index(row)

In [35]:
# fill NaN values
columns = data.columns

binary_columns = [2, 3, 9, 12, 13, 14, 15, 18, 19, 20, 21, 31, 32, 33]

categorical_columns = []
categorical_columns.extend(list(range(22,30)))
categorical_columns.extend(list(range(30,31)))
categorical_columns.extend(list(range(33,40)))

range_columns = [10, 4, 6, 7 , 8]
range_columns.extend(categorical_columns)


fill_binary_columns(binary_columns)
fill_categorical_columns(categorical_columns)
fill_range_columns(range_columns)

for c in range(len(columns)):
    if c not in binary_columns and c not in categorical_columns and c not in range_columns and data[columns[c]].dtype != object:
        data[columns[c]] = data[columns[c]].fillna(compute_mode(data[data.columns[c]])) 

for c in range_columns:
    data[columns[c]] = data[columns[c]].apply(categorical_to_integer, args=(c,))
    
#data["diagnosi"] = data["diagnosi"].apply(build_target_column)
data
original_data = data


In [36]:
data

,cod.,età (all'arruolamento),genere,menopausa,menarca (età),emicrania,diagnosi,lato,onset (età),familiarità,...,bmi,fumo si=1/no=0,alcool si=1/no=0,caffè si=1/no=0,oc/hrt si=1/no=0,familiarità per cardiopatia,ipertensione arteriosa,ima si=1/no=0,dislipidemia si=1/no=0,diabete si=1/no=0
0,BB12592,42.0,1,0.0,0,2,0,0,0,1.0,...,0,1.0,1.0,0,0,0,0,0,0,0
1,SOD001,42.0,1,0.0,1,3,1,1,1,1.0,...,0,1.0,1.0,0,0,0,0,0,0,0
2,SOD002,38.0,1,0.0,2,3,2,0,2,1.0,...,1,1.0,0.0,1,0,0,0,0,0,0
3,SOD003,44.0,0,0.0,0,3,2,0,3,1.0,...,0,0.0,0.0,1,0,1,0,0,0,0
4,SOD004,72.0,0,0.0,0,3,1,0,4,1.0,...,2,0.0,0.0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1034,255,57.0,1,1.0,0,1,5,1,35,1.0,...,24,1.0,1.0,0,0,0,0,0,0,0
1035,256,54.0,1,0.0,0,1,2,4,6,1.0,...,248,1.0,1.0,0,0,0,0,0,0,0
1036,257,32.0,1,0.0,0,1,2,1,0,1.0,...,249,1.0,1.0,0,0,0,0,0,0,0
1037,258,61.0,1,1.0,0,1,2,4,7,1.0,...,126,1.0,1.0,0,0,0,0,0,0,0


In [37]:
train_X, test_X, train_Y, test_Y = train_test_split(data.drop(["cod. ", "emicrania"], axis=1), data["emicrania"], test_size=0.3)

In [38]:
clf.fit(train_X, train_Y)

DecisionTreeClassifier(random_state=23)

In [39]:
preds = clf.predict(test_X)
preds

array([1, 1, 2, 1, 1, 1, 1, 4, 3, 1, 1, 1, 3, 1, 1, 1, 3, 1, 1, 1, 1, 3,
       3, 3, 1, 1, 3, 3, 2, 1, 2, 1, 3, 2, 3, 1, 1, 1, 2, 3, 1, 1, 1, 1,
       3, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 3, 3, 1, 2, 1, 1, 2, 1, 1, 1, 2,
       3, 3, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 4, 1, 1, 2, 3, 1, 1,
       1, 2, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 3,
       1, 1, 3, 2, 1, 3, 4, 1, 2, 2, 1, 1, 1, 3, 1, 3, 4, 3, 1, 1, 1, 3,
       1, 1, 1, 2, 1, 2, 1, 3, 1, 1, 1, 1, 2, 2, 1, 1, 3, 1, 3, 1, 3, 1,
       1, 1, 1, 1, 2, 1, 1, 3, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 3, 1, 1, 1, 1, 2, 2, 1, 2, 1, 3, 2, 3, 1, 3, 1, 2, 1,
       1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 3, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2,
       1, 1, 1, 1, 1, 3, 1, 1, 2, 3, 1, 1, 1, 1, 1, 3, 1, 2, 1, 1, 1, 2,
       1, 1, 2, 1, 3, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2,
       4, 2, 1, 1, 3, 1, 3, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 3, 2, 3, 3, 3,
       3, 2, 1, 1, 2, 1, 1, 1, 2, 3, 2, 1, 1, 1, 3,

In [40]:
accuracy = accuracy_score(preds, test_Y)
recall = recall_score(preds, test_Y, average="macro")
f1 = f1_score(preds, test_Y, average="macro")

C:\Users\semin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [41]:
print(f"Test Results:\n\nAccuracy: {accuracy}\nRecall: {recall}\nF1 measure: {f1}")

Test Results:

Accuracy: 0.9038461538461539
Recall: 0.7108070175438597
F1 measure: 0.6955019708999773


## Visualize the Decision Tree Logic

In [54]:
features = list(data.columns[1:].values)
features.pop(4)
features = [str(x) for x in features]
classes = [str(x) for x in CLASSES]

fig = plt.figure(figsize=(200,120))
_ = tree.plot_tree(clf, 
                   feature_names=features,  
                   class_names=classes,
                   filled=True)